# Introduction to Dash

In this part we will learn more about Dash syntax to build dashboards. We will combine it with what we've learnt so far about Plotly library in order to build an interactive dashboard with multiple charts.

## Dash Building Blocks

The first part of Dash app is App Layout that specifies the order and arrangement of all the components constituting a dashboard. The components can be of 2 types:

- Dash HTML components
- Dash Core components

### Dash HTML Components

Dash provides an abstraction layer to define all the necessary elements as Python objects without the need to use Javascript and HTML. Instead of writing HTML, you compose your layout using Python structures with the `dash_html_components` library. 

Here are some examples of the main HTML components:

![html](pics/html-comp.png)

### Dash Core Components

Dash Core components are supercharged interactive elements like Plotly charts, sliders, and dropdowns. A core set of components, written and maintained by the Dash team, is available in the `dash_core_components` library.

Here are some examples of the main Dash Core components:

![dcc](pics/dcc.png)

You can find full Dash documentation [here](https://dash.plot.ly/).

The format we will use in this part is sliglty different. Instead of executing the code directly from the cell, we will write the code to the python file using Jupyter notebook magic `%%writefile <filename.py>` and then run the file using another magic command `%run <filename.py>`. If you prefer you can also use your favourite IDE and write and execute the code directly from there.

In [1]:
%%writefile html_component.py
#static html example

# import necessary libraries
import dash
import dash_html_components as html

# select css stylesheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# initiate dash app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# define app layout
app.layout = html.Div([
    html.H1('Hello, PyDay!'),
    html.Div([
        html.P('Let\'s start learning about Dash HTML components by using some of them.'),
        html.P('Dash converts Python classes into HTML.'),
        html.P('So far we have used 5 HTML components: '),
        html.Ul([
            html.Li('H1 to create heading 1 tag'),
            html.Li('Div to create a <div> tag with group of elements'),
            html.Li('P to create <p> tags with paragraphs of text'),
            html.Li('Ul to create <ul> tags with unordered list'),
            html.Li('Li for listing each bullet point of the list')
        ])
    ])
])

# run dash app
if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting html_component.py


In [2]:
#%run html_component.py

Dash also supports Markdown, so as for the blocks of texts like in the example above we can re-write them using Dash core markdown component. 

In [3]:
%%writefile md_component.py
#static markdown example

# import necessary libraries
import dash
import dash_core_components as dcc

# select css stylesheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# initiate dash app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# define app layout
app.layout = dcc.Markdown('''
# Hello, PyDay!
We can write the same text as in the past example in Markdown: 

Let's start learning about Dash HTML components by using some of them.

Dash converts Python classes into HTML.

So far we have used **5 HTML components**:
- H1 to create heading 1 tag
- Div to create a `<div>` tag with group of elements
- P to create `<p>` tags with paragraphs of text
- Ul to create `<ul>` tags with unordered list
- Li for listing each bullet point of the list
''')

# run dash app
if __name__ == '__main__':
    app.run_server(debug=True)

Writing md_component.py


In [4]:
#%run md_component.py

Let's add some other Dash core components, such as graphs and dropdowns.

In [5]:
%%writefile dropdown.py

# import necessary libraries
import dash
import dash_html_components as html
import dash_core_components as dcc
import pandas as pd

# select css stylesheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# load the dataset
df = pd.read_csv('df_per_cap.csv')

# get the list of available countries for dropdown
available_countries = df.LOCATION.sort_values().unique()

# initiate dash app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# define app layout
app.layout = html.Div([
    html.H1('Meat Consumption'),
    dcc.Dropdown(
            id='countries-dropdown',
            options=[{'label': i, 'value': i} for i in available_countries],
            placeholder='Select country'
        )
])

# run dash app
if __name__ == '__main__':
    app.run_server(debug=True)

Writing dropdown.py


In [6]:
#%run dropdown.py

In [7]:
%%writefile total_line.py

# import necessary libraries
import dash
import dash_html_components as html
import dash_core_components as dcc
import pandas as pd
import plotly.graph_objs as go

# select css stylesheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# load the dataset and group by year
df = pd.read_csv('df_total.csv')
df_year = df.groupby('TIME').Value.sum().reset_index()

# get the list of available countries for dropdown
available_countries = df.LOCATION.sort_values().unique()

# initiate dash app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# define app layout
app.layout = html.Div([
    html.H1('Meat Consumption'),
    dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': i, 'value': i} for i in available_countries],
            placeholder='Select a country'
        ),
      dcc.Graph(
        id='total-line',
        figure={
            'data': [
                go.Scatter(
                    x=df_year.TIME,
                    y=df_year.Value,
                    mode='lines'
                )
            ],
            'layout': go.Layout(
                title=dict(text='Total Meat Consumption Evolution'),
                xaxis=dict(title='Year'),
                yaxis=dict(title='Meat Consumption, thousand tonnes')
            )
        }
     )
])

# run dash app
if __name__ == '__main__':
    app.run_server(debug=True)

Writing total_line.py


In [8]:
#%run total_line.py

## Dash App Callbacks: adding interactivity

Once we have defined the layout of the app, it's time for the fun part: adding interactivity! 

If our app was a flat, the **App Layout** would correspond to defining each room purpose and where all the furniture goes: 

![layout](pics/app_layout.jpg)

**Callabcks** part would correspond to making sure the flat actually works as expected: the lights go on and off, the heating is working if switched on, the water is running if we turn the tap on. 

![callbacks](pics/homer.gif)

In order to add interactivity, we will import Input and Output from `dash.dependencies`.

In [1]:
%%writefile total_line.py

# import necessary libraries
import dash
import dash_html_components as html
import dash_core_components as dcc
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# select css stylesheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# load the dataset 
df = pd.read_csv('df_per_cap.csv')

# get the list of available countries for dropdown
available_countries = df.LOCATION.sort_values().unique()

# initiate dash app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# define app layout
app.layout = html.Div([
    html.H1('Meat Consumption'),
    dcc.Dropdown(
            id='country-dropdown',
            options=[{'label': i, 'value': i} for i in available_countries],
            placeholder='Select a country',
            value='EU27'
        ),
      dcc.Graph(id='total-line')
])

# define inputs and outputs
@app.callback(
    Output('total-line', 'figure'),
    [Input('country-dropdown', 'value')])

# define function to update graph depending on selected country from the dropdown
def update_graph(country):
    if country:
        dff = df[df.LOCATION == country]
    else: 
        dff = df
    dff_year = dff.groupby('TIME').Value.sum().reset_index()
    
    fig = px.line(dff_year, x="TIME", y="Value", title='Meat Consumption Evolution')
    fig = fig.update_layout(xaxis_title='Year', yaxis_title='Meat consumption, kg per capita')
    
    return fig


# run dash app
if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting total_line.py


In [ ]:
#%run total_line.py

In [4]:
%%writefile total_line_and_map.py

# import necessary libraries
import dash
import dash_html_components as html
import dash_core_components as dcc
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

# select css stylesheet
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

# load the dataset 
df = pd.read_csv('df_per_cap.csv')

# get the list of available countries for dropdown
available_countries = df.LOCATION.sort_values().unique()

# initiate dash app
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# define app layout
app.layout = html.Div([
    html.H1('Meat Consumption', style={'margin-left': 20, 'margin-right': 20}),
    
    html.Div([
        html.Div([
            dcc.Dropdown(
                id='country-dropdown',
                options=[{'label': i, 'value': i} for i in available_countries],
                placeholder='Select a country',
                value='EU27'
            )
        ], style={'width': '48%', 'display': 'inline-block'}),

        html.Div([
            dcc.Slider(
                id='year-slider',
                min=df['TIME'].min(),
                max=df['TIME'].max(),
                value=df['TIME'].max(),
                marks={str(year): str(year) for year in df['TIME'].unique()}
            )
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
        
    ], style={'margin-left': 20, 'margin-right': 20}),
    
    html.Div([
        html.Div([
            dcc.Graph(id='total-line')
        ], style={'width': '48%', 'display': 'inline-block'}),
        
        html.Div([
            dcc.Graph(id='country-map')
        ], style={'width': '48%', 'float': 'right', 'display': 'inline-block'})
    ])
    
])

# define inputs and outputs
@app.callback(
    Output('total-line', 'figure'),
    [Input('country-dropdown', 'value')])

# define function to update line chart depending on selected country from the dropdown
def update_line_chart(country):
    if country:
        dff = df[df.LOCATION == country]
        dff_year = dff.groupby('TIME').Value.sum().reset_index()
    else: 
        dff = df
        dff_year = dff.groupby('TIME').mean.sum().reset_index()
        
    fig = px.line(dff_year, x='TIME', y='Value', title='Meat Consumption Evolution')
    fig = fig.update_layout(xaxis_title='Year', yaxis_title='Meat consumption, kg per capita')
    return fig

# define inputs and outputs
@app.callback(
    Output('country-map', 'figure'),
    [Input('year-slider', 'value')])

# define function to update map depending on selected year from the slider
def update_map(year):
    dff = df[df.TIME == year]
    dff_country = dff.groupby('LOCATION').Value.sum().reset_index()
    
    fig = px.choropleth(dff_country, 
                   locations="LOCATION", 
                   color="Value",
                   labels={'Value':'Kg per capita'},
                   title='Meat Consumption per Capita, 2018',
                   color_continuous_scale=px.colors.sequential.OrRd)
    fig = fig.update_layout(geo=dict(showframe=False))
    return fig

# run dash app
if __name__ == '__main__':
    app.run_server(debug=True)

Overwriting total_line_and_map.py


In [2]:
#%run total_line_and_map.py

## PRACTICE TIME

Create a Dash dashboard answering the following questions: 

- What is the trend of total meat consumptrion per capita over years for each country?
- What is the trend of consumption of each type of meat per capita over years for each country?
- Which countries consume the most and the least amount of meat per capita, given a specific year (and meat type)?
- Is there any relationship between meat consumption per capita and GDP per capita?